# Develop MLPs for Time Series Forecasting

Multilayer Perceptrons, or MLPs for short, can be applied to time series forecasting. A challenge with using MLPs for time series forecasting is in the preparation of the data. Specifically, lag observations must be flattened into feature vectors. 

In this tutorial, you will discover how to develop a suite of MLP models for a range of standard time series forecasting problems. The objective of this tutorial is to provide standalone examples of each model on each type of time series problem as a template that you can copy and adapt for your specific time series forecasting
problem. 

We will discover how to develop a suite of Multilayer Perceptron models for a range of standard time series forecasting problems.

In [34]:
from numpy import array
from numpy import hstack

In [35]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [36]:
# define input sequence
raw_seq = [i*10 for i in range(1, 20)]
print(raw_seq)

# choose a number of time steps
n_steps = 3

# split into samples
X, y = split_sequence(raw_seq, n_steps)

# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190]
[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90
[70 80 90] 100
[ 80  90 100] 110
[ 90 100 110] 120
[100 110 120] 130
[110 120 130] 140
[120 130 140] 150
[130 140 150] 160
[140 150 160] 170
[150 160 170] 180
[160 170 180] 190


In [37]:
# univariate mlp example
from numpy import array
from keras.models import Sequential
from keras.layers import Dense

In [38]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [39]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

In [40]:
# demonstrate prediction
x_input = array([140, 150, 160])
x_input = x_input.reshape((1, n_steps))
print(x_input)
yhat = model.predict(x_input)
print(yhat)

[[140 150 160]]
[[169.93585]]


## Multivariate MLP Models

Multivariate time series data means data where there is more than one observation for each time step. There are two main models that we may require with multivariate time series data

### Multiple Input Series

In [41]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])
out_seq

array([ 25,  45,  65,  85, 105, 125, 145, 165, 185])

In [42]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [43]:
# split a multivariate sequence into samples
def split_sequence(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix, :-1], 
        seq_y = sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [32]:
# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequence(dataset, n_steps)
print(X.shape, y.shape)
# Summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 1, 3, 2) (7,)
[[[10 15]
  [20 25]
  [30 35]]] 65
[[[20 25]
  [30 35]
  [40 45]]] 85
[[[30 35]
  [40 45]
  [50 55]]] 105
[[[40 45]
  [50 55]
  [60 65]]] 125
[[[50 55]
  [60 65]
  [70 75]]] 145
[[[60 65]
  [70 75]
  [80 85]]] 165
[[[70 75]
  [80 85]
  [90 95]]] 185


## MLP Model
Before we can fit an MLP on this data, we must flatten the shape of the input samples. MLPs require that the shape of the input portion of each sample is a vector. With a multivariate input, we will have multiple vectors, one for each time step. We can flatten the temporal structure of each input sample

First, we can calculate the length of each input vector as the number of time steps multiplied
by the number of features or time series. We can then use this vector size to reshape the input.

In [33]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], -1))
print(X.shape)
X

(7, 6)


array([[10, 15, 20, 25, 30, 35],
       [20, 25, 30, 35, 40, 45],
       [30, 35, 40, 45, 50, 55],
       [40, 45, 50, 55, 60, 65],
       [50, 55, 60, 65, 70, 75],
       [60, 65, 70, 75, 80, 85],
       [70, 75, 80, 85, 90, 95]])

We can now define an MLP model for the multivariate input where the vector length is used
for the input dimension argument

In [ ]:
# define model
model